# 1.COVID-19 graph embedding alone

## 1.1. ranking model

In [9]:
import numpy as np
import pandas as pd
import time
import re
import math
import random
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.modules import Module
from torch.utils.data import Dataset, DataLoader

In [2]:
data_path='data/'
exp_id='v0'
device_id='cpu' #'cpu' if CPU, device number if GPU

In [11]:
device=torch.device(device_id)

In [3]:
le=pickle.load(open(data_path+'LabelEncoder_'+exp_id+'.pkl', 'rb'))
edge_index=pickle.load(open(data_path+'edge_index_'+exp_id+'.pkl','rb'))

In [4]:
types=np.array([item.split('_')[0] for item in le.classes_ ])

In [5]:
#label
trials=pd.read_excel(data_path+'literature-mining/All_trails_5_24.xlsx',header=1,index_col=0)
trials_drug=set([drug.strip().upper() for lst in trials.loc[trials['study_category'].apply(lambda x: 'drug' in x.lower()),'intervention'].apply(lambda x: re.split(r'[+|/|,]',x.replace(' vs. ', '/').replace(' vs ', '/').replace(' or ', '/').replace(' with and without ', '/').replace(' /wo ', '/').replace(' /w ', '/').replace(' and ', '/').replace(' - ', '/').replace(' (', '/').replace(') ', '/'))).values for drug in lst])
drug_labels=[1 if drug.split('_')[1] in trials_drug else 0 for drug in le.classes_[types=='drug'] ]

In [54]:
z_np = pickle.load(open(data_path+'COVID_embedding_'+exp_id+'.pkl', 'rb'))

In [55]:
seed=70
indices = np.arange(len(drug_labels))
X_train, X_test, y_train, y_test,indices_train,indices_test=train_test_split(z_np[types=='drug'],drug_labels,indices, test_size=0.5,random_state=seed,)

In [56]:
#Variable wrapping for torch.tensor
_X_train, _y_train=Variable(torch.tensor(X_train,dtype=torch.float).to(device)), Variable(torch.tensor(y_train,dtype=torch.float).to(device))
_X_test, _y_test=Variable(torch.tensor(X_test,dtype=torch.float).to(device)), Variable(torch.tensor(y_test,dtype=torch.float).to(device))

In [14]:
class Classifier(nn.Module):
    def __init__(self,embedding_dim):
        super(Classifier, self).__init__() 
        self.fc1=nn.Linear(embedding_dim,embedding_dim)
        self.fc2=nn.Linear(embedding_dim,1)
        self.bn=nn.BatchNorm1d(embedding_dim)
    def forward(self, x):
        residual1 = x
        x = F.dropout(x, training=self.training)
        x= self.bn(F.dropout(F.relu(self.fc1(x)),training=self.training))
        x += residual1  
        return self.fc2(x)

In [15]:
from torch.utils.data import BatchSampler, WeightedRandomSampler
class BPRLoss(nn.Module):
    def __init__(self, num_neg_samples):
        super(BPRLoss, self).__init__()
        self.num_neg_samples=num_neg_samples
    
    def forward(self, output, label):
        positive_output=output[label==1]
        negative_output=output[label!=1]
        
        #negative sample proportional to the high values
        negative_sampler=WeightedRandomSampler(negative_output-min(negative_output), num_samples=self.num_neg_samples*len(positive_output),replacement=True)
        negative_sample_output=negative_output[torch.tensor(list(BatchSampler(negative_sampler, batch_size=len(positive_output),drop_last=True)),dtype=torch.long).t()]
        return -(positive_output.view(-1,1)-negative_sample_output).sigmoid().log().mean()

In [57]:
clf=Classifier(embedding_dim=128).to(device)
optimizer=torch.optim.Adam(clf.parameters())
criterion=BPRLoss(num_neg_samples=15)

In [58]:
best_auprc=0
for epoch in range(30):
    clf.train()
    optimizer.zero_grad()
    out = clf(_X_train)
    loss=criterion(out.squeeze(), _y_train)
    loss.backward()
    optimizer.step()   
    print('training loss',loss.item())

    clf.eval()
    print('test loss', criterion(clf(_X_test).squeeze(), _y_test).item())
    prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
    auprc=metrics.average_precision_score(y_test,prob)
    if auprc>best_auprc:
        best_auproc=auprc
        torch.save(clf, data_path+'nn_clf_covid_embedding.pt')

training loss 0.7957777380943298
test loss 0.6983648538589478
training loss 0.8004921078681946
test loss 0.6988610625267029
training loss 0.7474350333213806
test loss 0.6995164752006531
training loss 0.8172909617424011
test loss 0.7001990079879761
training loss 0.840722382068634
test loss 0.7008649706840515
training loss 0.7738116979598999
test loss 0.7015867829322815
training loss 0.8056227564811707
test loss 0.7023756504058838
training loss 0.8166890740394592
test loss 0.7031198740005493
training loss 0.7883610725402832
test loss 0.7038981914520264
training loss 0.8199113011360168
test loss 0.7047076225280762
training loss 0.8332513570785522
test loss 0.7057130336761475
training loss 0.8371492624282837
test loss 0.7067953944206238
training loss 0.8037989139556885
test loss 0.7077513337135315
training loss 0.763800859451294
test loss 0.7088028788566589
training loss 0.7703692317008972
test loss 0.7099213600158691
training loss 0.8040730953216553
test loss 0.7110167145729065
training l

In [59]:
clf.load_state_dict(torch.load(data_path+'nn_clf_covid_embedding.pt').state_dict())

<All keys matched successfully>

In [60]:
#Compute AUC
clf.eval()

prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
print("AUROC", metrics.roc_auc_score(y_test,prob))
print("AUPRC", metrics.average_precision_score(y_test,prob))

AUROC 0.5068115192274529
AUPRC 0.04367524330005571


## 1.2. baseline models

In [61]:
clf=LogisticRegression().fit(X_train,y_train)
print("Logit AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("Logit AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

Logit AUROC 0.4619526877674833
Logit AUPRC 0.09362296208531114


In [62]:
clf=GradientBoostingClassifier().fit(X_train,y_train)
print("XGBoost AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("XGBoost AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

XGBoost AUROC 0.4992475191647463
XGBoost AUPRC 0.04322014411352604


In [63]:
clf=RandomForestClassifier().fit(X_train,y_train)
print("rf AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("rf AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

rf AUROC 0.4992475191647463
rf AUPRC 0.04322014411352604


In [64]:
clf=make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True)).fit(X_train,y_train)
print("svm AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("svm AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

svm AUROC 0.36704604242110705
svm AUPRC 0.07614197571740616


# 2. DRKG embedding alone

## 2.1. ranking model

In [43]:
z_np = pickle.load(open(data_path+'node_feature_'+exp_id+'.pkl','rb'))

In [44]:
seed=70
indices = np.arange(len(drug_labels))
X_train, X_test, y_train, y_test,indices_train,indices_test=train_test_split(z_np[types=='drug'],drug_labels,indices, test_size=0.5,random_state=seed,)

In [45]:
#Variable wrapping for torch.tensor
_X_train, _y_train=Variable(torch.tensor(X_train,dtype=torch.float).to(device)), Variable(torch.tensor(y_train,dtype=torch.float).to(device))
_X_test, _y_test=Variable(torch.tensor(X_test,dtype=torch.float).to(device)), Variable(torch.tensor(y_test,dtype=torch.float).to(device))

In [46]:
clf=Classifier(embedding_dim=400).to(device)
optimizer=torch.optim.Adam(clf.parameters())
criterion=BPRLoss(num_neg_samples=15)

In [47]:
best_auprc=0
for epoch in range(30):
    clf.train()
    optimizer.zero_grad()
    out = clf(_X_train)
    loss=criterion(out.squeeze(), _y_train)
    loss.backward()
    optimizer.step()   
    print('training loss',loss.item())

    clf.eval()
    print('test loss', criterion(clf(_X_test).squeeze(), _y_test).item())
    prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
    auprc=metrics.average_precision_score(y_test,prob)
    if auprc>best_auprc:
        best_auproc=auprc
        torch.save(clf, data_path+'nn_clf_DRKG_embedding.pt')

training loss 0.7395689487457275
test loss 0.616218090057373
training loss 0.4780997633934021
test loss 0.5486316084861755
training loss 0.3808656632900238
test loss 0.5048272013664246
training loss 0.27103176712989807
test loss 0.4665544927120209
training loss 0.309094101190567
test loss 0.4589807391166687
training loss 0.2699897885322571
test loss 0.460541695356369
training loss 0.24963025748729706
test loss 0.4448689818382263
training loss 0.3198316991329193
test loss 0.4421408176422119
training loss 0.2551022469997406
test loss 0.43925824761390686
training loss 0.3009866774082184
test loss 0.4519835114479065
training loss 0.19218678772449493
test loss 0.4679725766181946
training loss 0.24697791039943695
test loss 0.4696771800518036
training loss 0.26026809215545654
test loss 0.44761529564857483
training loss 0.1606442779302597
test loss 0.4597027897834778
training loss 0.19212880730628967
test loss 0.4595140814781189
training loss 0.2429376095533371
test loss 0.47952720522880554
tr

In [48]:
clf.load_state_dict(torch.load(data_path+'nn_clf_DRKG_embedding.pt').state_dict())

<All keys matched successfully>

In [49]:
#Compute AUC
clf.eval()

prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
print("AUROC", metrics.roc_auc_score(y_test,prob))
print("AUPRC", metrics.average_precision_score(y_test,prob))

AUROC 0.8667481854238193
AUPRC 0.18218957502766814


## 2.2. baseline models

In [50]:
clf=LogisticRegression().fit(X_train,y_train)
print("Logit AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("Logit AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

Logit AUROC 0.8436015613977331
Logit AUPRC 0.18394788718419244


In [51]:
clf=GradientBoostingClassifier().fit(X_train,y_train)
print("XGBoost AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("XGBoost AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

XGBoost AUROC 0.8554609728950132
XGBoost AUPRC 0.13820130901483108


In [52]:
clf=RandomForestClassifier().fit(X_train,y_train)
print("rf AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("rf AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

rf AUROC 0.855993980153318
rf AUPRC 0.13835021652961998


In [53]:
clf=make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True)).fit(X_train,y_train)
print("svm AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("svm AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

svm AUROC 0.8842276881594006
svm AUPRC 0.2578582442037748


# 3. hybrid embedding

## 3.1. ranking model

In [31]:
z_np = pickle.load(open(data_path+'hybrid_embedding_'+exp_id+'.pkl','rb'))

In [32]:
seed=70
indices = np.arange(len(drug_labels))
X_train, X_test, y_train, y_test,indices_train,indices_test=train_test_split(z_np[types=='drug'],drug_labels,indices, test_size=0.5,random_state=seed,)

In [33]:
#Variable wrapping for torch.tensor
_X_train, _y_train=Variable(torch.tensor(X_train,dtype=torch.float).to(device)), Variable(torch.tensor(y_train,dtype=torch.float).to(device))
_X_test, _y_test=Variable(torch.tensor(X_test,dtype=torch.float).to(device)), Variable(torch.tensor(y_test,dtype=torch.float).to(device))

In [35]:
clf=Classifier(embedding_dim=128).to(device)
optimizer=torch.optim.Adam(clf.parameters())
criterion=BPRLoss(num_neg_samples=15)

In [36]:
best_auprc=0
for epoch in range(30):
    clf.train()
    optimizer.zero_grad()
    out = clf(_X_train)
    loss=criterion(out.squeeze(), _y_train)
    loss.backward()
    optimizer.step()   
    print('training loss',loss.item())

    clf.eval()
    print('test loss', criterion(clf(_X_test).squeeze(), _y_test).item())
    prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
    auprc=metrics.average_precision_score(y_test,prob)
    if auprc>best_auprc:
        best_auproc=auprc
        torch.save(clf, data_path+'nn_clf_hybrid_embedding.pt')

training loss 0.9536146521568298
test loss 0.7626969218254089
training loss 0.7878918647766113
test loss 0.7334699630737305
training loss 0.7278525233268738
test loss 0.705784261226654
training loss 0.6320992708206177
test loss 0.678355872631073
training loss 0.5433192253112793
test loss 0.6581498980522156
training loss 0.48433196544647217
test loss 0.6377235054969788
training loss 0.5341165661811829
test loss 0.6118820309638977
training loss 0.49445411562919617
test loss 0.6034524440765381
training loss 0.48542895913124084
test loss 0.5843536853790283
training loss 0.4485393464565277
test loss 0.5750580430030823
training loss 0.49111926555633545
test loss 0.557050347328186
training loss 0.41423916816711426
test loss 0.5427328944206238
training loss 0.481771320104599
test loss 0.5426845550537109
training loss 0.3665356934070587
test loss 0.5306793451309204
training loss 0.4309069812297821
test loss 0.5272321701049805
training loss 0.40130066871643066
test loss 0.5076251029968262
traini

In [37]:
clf.load_state_dict(torch.load(data_path+'nn_clf_hybrid_embedding.pt').state_dict())

<All keys matched successfully>

In [38]:
#Compute AUC
clf.eval()

prob=torch.sigmoid(clf(_X_test)).cpu().detach().numpy().squeeze()
print("AUROC", metrics.roc_auc_score(y_test,prob))
print("AUPRC", metrics.average_precision_score(y_test,prob))

AUROC 0.8930654188026148
AUPRC 0.27073473253095137


## 3.2. baseline models

In [39]:
clf=LogisticRegression().fit(X_train,y_train)
print("Logit AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("Logit AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

Logit AUROC 0.9045877815924376
Logit AUPRC 0.3189766638941174


In [40]:
clf=GradientBoostingClassifier().fit(X_train,y_train)
print("XGBoost AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("XGBoost AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

XGBoost AUROC 0.8656508175390742
XGBoost AUPRC 0.20530366246464798


In [41]:
clf=RandomForestClassifier().fit(X_train,y_train)
print("rf AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("rf AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

rf AUROC 0.8670186082239885
rf AUPRC 0.252456982482454


In [42]:
clf=make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True)).fit(X_train,y_train)
print("svm AUROC", roc_auc_score(y_test,clf.predict_proba(X_test)[:,1]))
print("svm AUPRC", average_precision_score(y_test,clf.predict_proba(X_test)[:,1]))

svm AUROC 0.8549083697816238
svm AUPRC 0.26104412389180137
